#### Import

In [3]:
import os
os.environ["CUDA_DEVICE_ORDER"] = "PCI_BUS_ID"
os.environ["CUDA_VISIBLE_DEVICES"] = "5"

In [4]:
import pandas as pd, numpy as np
import dash
# from dash import html
from plotly.colors import sequential
from holoviews.element.tiles import CartoDark
import cudf, cupy
import sys,os,datetime,random
import geopandas as gpd
from shapely.geometry import Point
# pd.set_option('display.float_format','{:.0f}'.format)
from dash import Dash, dcc, html
import numpy as np
import pandas as pd
from dash.dependencies import Input, Output, State
# import plotly.express as px
# import plotly.graph_objects as go
import holoviews as hv
from holoviews.plotting.plotly.dash import to_dash
from holoviews.operation.datashader import datashade,rasterize,dynspread,shade
from holoviews import streams
import datashader.transfer_functions as tf
import datashader as ds
from pyproj import Transformer
from holoviews import streams
from holoviews import opts
from holoviews.streams import Stream, param, LinkedStream
import panel as pn
import dask_cudf
import colorcet as cc

In [5]:
# data_path = 'data/census_migration_data.parquet'
# df = dask_cudf.read_parquet(data_path)

In [6]:
# df = df.sample(frac=0.05)
# df.head(10)

In [7]:
# df = df.rename(columns={'ID20':'blockid','STATE':'state','COUNTY':'county','P_delta':'block_diff','P_net':'block_net'})
# df.sample(frac=0.10).head(10)

In [8]:
# len(df)

In [9]:
# df = df.compute()

In [10]:
# df.to_csv('data/census_migration_small.csv')

In [6]:
# df = pd.read_csv('data/census_migration_small.csv',encoding='unicode_escape',dtype={'county':'str'})
df = cudf.read_csv('../data/final_data_full.csv',usecols=['ID20','x','y','COUNTY','P_delta','P_net'])
# df.to_pandas()
df.head()

,ID20,x,y,COUNTY,P_delta,P_net
0,10010201001000,-86.481384,32.471474,Autauga County,-9.5,-1
1,10010201001000,-86.478294,32.471313,Autauga County,-9.5,-1
2,10010201001000,-86.479706,32.471794,Autauga County,-9.5,-1
3,10010201001000,-86.477780,32.469050,Autauga County,-9.5,-1
4,10010201001000,-86.481580,32.472637,Autauga County,-9.5,-1


In [8]:
df.rename(columns={'ID20':'blockid', 'P_net':'block_net','COUNTY':'county'},inplace=True)

In [9]:
# df = cudf.from_pandas(df)

#### Panel Dashboard

In [10]:
def race_bar(ds,selected_race): #
    if selected_race==0:
        agg_df =filtered_df[filtered_df.block_net!=0].groupby('block_net')['blockid'].count().reset_index()
        labels = hv.Dimension('block_net', values = ['Out','In'])
        return hv.Bars(data=agg_df,kdims=['block_net'],vdims=['blockid'])
    else:
        agg_df =filtered_df[filtered_df.race.abs()==selected_race].groupby('race')['blockid'].count().reset_index()
        labels = hv.Dimension('race', values = ['Out','In'])
        return hv.Bars(data=agg_df,kdims=['race'],vdims=['blockid'])

def bars(ds,limit,selected_race):
    # print("Entered bar")   
    # print(limit)
    # print(len(ds))
    # print(index)
    # print(kdims)
    # filtered_df = ds.data
    if selected_race==0:
        agg_df = filtered_df.groupby('county')['blockid'].count().sort_values(ascending=False).reset_index()[:limit]
        kdim_county = hv.Dimension('county', values = agg_df.to_pandas()['county'].tolist()[:limit])
        return hv.Bars(data=agg_df,kdims=kdim_county,vdims=['blockid'])#.opts(color='x',cmap=cmap_selector.value)
    else:
        agg_df = filtered_df[filtered_df.race.abs()==selected_race].groupby('county')['blockid'].count().sort_values(ascending=False).reset_index()[:limit]
        kdim_county = hv.Dimension('county', values = agg_df.to_pandas()['county'].tolist()[:limit])
        return hv.Bars(data=agg_df,kdims=kdim_county,vdims=['blockid'])    

def filter_dataset(bounds,boundsx,b2): 
    # print("Entered data filter")
    # print(bounds,boundsx,b2)
    
    global bounds_prev
    global boundsx_prev
    global b2_prev
    global filtered_df
    global c
    c+=1
    if (bounds_prev!=bounds) &  (bounds is not None):
        bounds_prev = bounds
        x0,y0,x1,y1 = bounds
        filtered_df = filtered_df[(filtered_df.easting>=x0) & (filtered_df.easting<=x1) & (filtered_df.northing<=y0) & (filtered_df.northing>=y1)]
        
    if (boundsx_prev!=boundsx) &  (boundsx is not None):
        boundsx_prev = boundsx
        x_min, x_max = boundsx
        filtered_df = filtered_df[(filtered_df.block_diff>=x_min) & (filtered_df.block_diff<=x_max)]
          
    if (b2_prev!=b2) &  (b2 is not None) :
        b2_prev = b2  
        agg_df = filtered_df.groupby('county')['blockid'].count().sort_values(ascending=False).reset_index()
        id_min, id_max = b2
        id_min,id_max = max(0,round(id_min)), round(id_max)
        # print(id_min,id_max+1)
        
        agg_df = agg_df.iloc[id_min:id_max+1]
        # print(selected_bar)
        selected_counties = agg_df.to_pandas().county.tolist()
        filtered_df = filtered_df[filtered_df['county'].isin(selected_counties)] # filter dataset to keep selected counties
    
    return hv.Dataset(filtered_df)

def scatter(ds,selected_race):
    # print("Entered scatter")
    # print(len(ds))
    # print(view_selection)
    filtered_df = ds.data
    if len(filtered_df)==0:
        return hv.Points(df[:1], ["easting", "northing"],vdims='block_net').options(color='block_net')  # Empty graph
    
    if selected_race == 0:
        return hv.Points(filtered_df, ["easting", "northing"],vdims='block_net').options(color='block_net') 
    else: return hv.Points(filtered_df[filtered_df.race.abs()==selected_race], ["easting", "northing"],vdims='block_net').options(color='block_net')       

def histogram(ds,selected_race):
    # print("Entered histogram")
    # print(len(ds))
    filtered_df = ds.data
    if selected_race==0:
        return hv.Histogram(np.histogram(filtered_df['block_diff'].to_pandas().values,bins=100))
    else:
        return hv.Histogram(np.histogram(filtered_df[filtered_df.race.abs()==selected_race]['block_diff'].to_pandas().values,bins=100))

def set_dragmode(plot, element):
    fig = plot.state
    fig['layout']['dragmode'] = "select"
    if isinstance(element, hv.Histogram):
        # Constrain histogram selection direction to horizontal
        fig['layout']['selectdirection'] = "h"
        fig['layout']['uirevision'] =False

bounds_prev = None
boundsx_prev = None
b2_prev = None
c= 0
filtered_df = df.copy()


tiles = CartoDark()
agg_df = filtered_df.groupby('county')['blockid'].count().sort_values(ascending=False).reset_index()
map_stream_box = streams.BoundsXY() # stream1
hist_stream_bx = streams.BoundsX()# stream 2
bar_stream_sel = streams.BoundsX().rename(boundsx='b2') # stream3

#widgets area
number = hv.Dimension('limit',values=[1,2,3,4])
dropdown = hv.Dimension('Order',values=['Top','Bottom'])
cmaps  = {n: cc.palette[n] for n in ['kbc', 'fire', 'bgy', 'bgyw', 'bmy', 'gray', 'kbc']}
cmap_selector = pn.widgets.Select(name='Colormap', options=cmaps)
race_options = {'All':0,'White':1, 'African American':2,'American Indian and Alaska Native':3,'Asian':4,'Native Hawaiian and Other Pacific Islander':5,'Some Other Race':6,'Two or More Races':7 }
race_selector = pn.widgets.Select(name='Race', options=race_options)

filtered_ds = hv.DynamicMap(filter_dataset,streams=[map_stream_box,hist_stream_bx,bar_stream_sel])#.redim.range(limit=(3,7)) #kdims=[number]

# filtered_ds = hv.DynamicMap(filter_dataset,kdims=number,streams=[map_stream_box,hist_stream_bx,bar_stream_sel])
dynamic_points = filtered_ds.apply(scatter,selected_race=race_selector)
dynamic_img = datashade(dynamic_points,cmap=cmap_selector)
dynamic_histogram = filtered_ds.apply(histogram,selected_race=race_selector) 
limit_slider = pn.widgets.IntSlider(name='Limit', value=10, start=5, end=50)
dynamic_bar = filtered_ds.apply(bars,limit=limit_slider,selected_race=race_selector)#.redim.range(time=(3,7))
dynamic_race = filtered_ds.apply(race_bar,selected_race=race_selector) #

dynamic_img.opts(opts.Scatter(hooks=[set_dragmode]))
dynamic_histogram.opts(opts.Histogram(hooks=[set_dragmode]))
dynamic_bar.opts(opts.Bars(hooks=[set_dragmode]))
dynamic_race.opts(opts.Bars(hooks=[set_dragmode]))



map_stream_box.source = dynamic_img
hist_stream_bx.source = dynamic_histogram
bar_stream_sel.source = dynamic_bar

# print(filtered_ds)
# print(dynamic_bar)

# map_overlay,hist_overlay,bar_overlay = (tiles*dynamic_points*img), (hist_base*dynamic_histogram), dynamic_bar #(agg_hist_base*dynamic_agg_histogram)
# print(overlay)
map_overlay,hist_overlay,bar_overlay,race_overlay = tiles*dynamic_img, dynamic_histogram , dynamic_bar,dynamic_race
# layout1 = (map_overlay.opts(height=600,width=800),hist_overlay + bar_overlay)

panel_dashboard = pn.Column(
                           pn.Row(pn.layout.Spacer(width=30),cmap_selector,race_selector),
                           pn.Row(map_overlay.opts(width=1000),race_overlay.opts(width=1000)),
                           pn.Row(pn.layout.Spacer(width=1100),limit_slider),
                           pn.Row(dynamic_histogram.opts(width =1000),dynamic_bar.opts(width=1000))
) #,race_overlay.opts(width=1000)

In [11]:
panel_dashboard.servable(title='Census Migration Demo')

Invoked as dynamic_operation(selected_race=0)
Invoked as dynamic_operation(height=400, scale=1.0, width=400, x_range=None, y_range=None, cmap=['#00004e', '#000150', '#000152', '#000154', '#000255', '#000257', '#000259', '#00025b', '#00025d', '#00025f', '#010261', '#010263', '#020265', '#030267', '#030269', '#04026c', '#05026e', '#060270', '#060272', '#070274', '#080276', '#090178', '#09017a', '#0a017c', '#0b017e', '#0b0181', '#0c0183', '#0d0185', '#0d0187', '#0e0189', '#0e018b', '#0f018e', '#0f0190', '#0f0192', '#100194', '#100197', '#100199', '#10019b', '#10019d', '#1001a0', '#0f01a2', '#0f02a4', '#0e02a7', '#0e02a9', '#0d02ab', '#0c03ae', '#0b03b0', '#0a04b2', '#0a04b5', '#0904b7', '#0905b9', '#0806bb', '#0806bd', '#0707bf', '#0707c2', '#0708c4', '#0709c6', '#070ac8', '#070bca', '#080ccc', '#080dce', '#090dd0', '#0a0ed1', '#0b0fd3', '#0c10d5', '#0d11d7', '#0e12d9', '#0f13db', '#1014dc', '#1215de', '#1316e0', '#1417e1', '#1618e3', '#1719e5', '#181ae6', '#1a1be7', '#1b1ce9', '#1c1eea',

DataError: Supplied data does not contain specified dimensions, the following dimensions were not found: ['easting', 'northing']

cuDFInterface expects tabular data, for more information on supported datatypes see http://holoviews.org/user_guide/Tabular_Datasets.html

Column
    [0] Row
        [0] Spacer(width=30)
        [1] Select(name='Colormap', options={'kbc': ['#00004e', ...}, value=['#00004e', '#000150', ...])
        [2] Select(name='Race', options={'All': 0, 'White': 1, ...}, value=0)
    [1] Row
        [0] HoloViews(DynamicMap)
        [1] HoloViews(DynamicMap)
    [2] Row
        [0] Spacer(width=1100)
        [1] IntSlider(end=50, name='Limit', start=5, value=10)
    [3] Row
        [0] HoloViews(DynamicMap)
        [1] HoloViews(DynamicMap)